In [134]:
import pandas as pd
import re
import numpy as np
from tqdm import tqdm
from nltk import word_tokenize
from fuzzywuzzy import fuzz
from collections import Counter
from string import punctuation
from string import digits
from stop_words import get_stop_words
from nltk.corpus import stopwords
from fuzzywuzzy import process

In [135]:
punct = punctuation+'©«»—…“”*№– ' + digits
stop_words_list = get_stop_words('ru')
stop_words_list2 = stopwords.words('russian')
newlist = stop_words_list + stop_words_list2
for word in ["свой"]:
    newlist.append(word)
newlist_set = set(newlist)

In [136]:
df = pd.read_csv('result_tar.gz', compression='gzip', sep='\t', error_bad_lines=False)

b'Skipping line 37039: expected 12 fields, saw 15\nSkipping line 45611: expected 12 fields, saw 15\n'
b'Skipping line 354339: expected 12 fields, saw 18\nSkipping line 360804: expected 12 fields, saw 102\nSkipping line 360819: expected 12 fields, saw 78\nSkipping line 379126: expected 12 fields, saw 14\nSkipping line 444446: expected 12 fields, saw 16\nSkipping line 449079: expected 12 fields, saw 16\nSkipping line 450250: expected 12 fields, saw 15\n'
b'Skipping line 481374: expected 12 fields, saw 18\nSkipping line 510619: expected 12 fields, saw 20\nSkipping line 512271: expected 12 fields, saw 14\nSkipping line 514317: expected 12 fields, saw 15\nSkipping line 519707: expected 12 fields, saw 40\n'
b'Skipping line 538495: expected 12 fields, saw 13\n'
b'Skipping line 765901: expected 12 fields, saw 56\nSkipping line 773310: expected 12 fields, saw 43\n'
b'Skipping line 805899: expected 12 fields, saw 13\nSkipping line 868492: expected 12 fields, saw 21\nSkipping line 871801: expecte

In [137]:
df = df.drop(["url", "video_url", "source_name", "author_name"], axis=1)
df = df.dropna()
df = df.drop_duplicates()
df = df.reset_index(drop=True)

In [138]:
def which_language (text):
    if bool(re.search('[\u0400-\u04FF]', text)) == True:
        return 'russian'
    else:
        return 'english'
    
lang=[]
for i,text in enumerate(df['result.tsv'].values): 
    lang.append(which_language(str(text)))
    
df['Language'] = lang
grouped=df.groupby('Language')
for name, group in grouped: 
    if name == 'english': 
        english = group.drop('Language',axis=1) 
    else: 
        russian = group.drop('Language',axis=1)


In [139]:
unique_leagues = russian['league_name_in_russian'].unique().tolist()
len(unique_leagues)

288

In [140]:
unique_teams1 = russian['team_of_season_1_name_in_russian'].unique().tolist()
len(unique_teams1)

3773

In [141]:
unique_teams2 = russian['team_of_season_2_name_in_russian'].unique().tolist()
len(unique_teams2)

3730

In [142]:
unique_teams = unique_teams1 + unique_teams2
unique_teams_set = set(unique_teams)
len(unique_teams_set)

4422

In [143]:
russian = pd.read_csv('russian_lists_and_directmatch.csv')

In [144]:
russian = russian[russian['result.tsv'].map(len) > 30]

In [145]:
russian = russian.reset_index(drop=True)

In [146]:
russian.shape

(524733, 10)

In [147]:
russian.head(3)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,list_direct_match_for_teams1,list_direct_match_for_teams2
0,Сборная Турции продолжает свою дорогу на Чемпи...,1.65,"- 1,5",Олимп,2019-06-02 18:00:00,Международные товарищеские матчи,Турция,Узбекистан,0.0,1.0
1,Двадцать девятого числа состоится один весьма ...,1.52,"– тотал меньше 2,5",Winline,2019-06-28 23:20:00,Кубок Америки,Колумбия,Чили,0.0,0.0
2,АЗ Алкмар В первом матче голландский клуб созд...,1.85,АЗ Алкмар забьет в обоих таймах,Лига Ставок,2019-08-15 18:30:00,Лига Европы УЕФА,АЗ Алкмаар,Мариуполь,0.0,0.0


In [148]:
subset_russian = russian.sample(n=10000)
subset_russian.head(5)


,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,list_direct_match_for_teams1,list_direct_match_for_teams2
8190,Леганес в этом сезоне неплохо умеет выжимать р...,2.340,1X,Леон,2019-04-15 19:00:00,Примера,Леганес,Реал Мадрид,1.0,0.0
440344,"Сегодня состоится матч по Футболу, в 2 - Дивиз...",1.480,Ф1 (0),1xСтавка,2019-04-20 16:00:00,Сегунда,Альбасете,Лас-Пальмас,0.0,0.0
291808,Квалификация Евро. Финляндия против Италии. Ду...,1.720,П2,1xСтавка,2019-09-08 18:45:00,Чемпионат Европы. Квалификация,Финляндия,Италия,0.0,0.0
234610,"Арсенал дома, команда сильнее и увереннее",1.444,П1,Winline,2019-09-22 15:30:00,Премьер-лига,Арсенал,Астон Вилла,0.0,0.0
433349,барселона проиграла лигу чемпионов она шас буд...,4.100,Ф1 (-2),Леон,2019-05-25 19:00:00,Кубок Испании,Барселона,Валенсия,0.0,0.0


In [149]:
subset_russian = subset_russian.reset_index(drop=True)
subset_russian.head(5)

,result.tsv,odd,tip_name,bookmaker_name,match_date_time,league_name_in_russian,team_of_season_1_name_in_russian,team_of_season_2_name_in_russian,list_direct_match_for_teams1,list_direct_match_for_teams2
0,Леганес в этом сезоне неплохо умеет выжимать р...,2.340,1X,Леон,2019-04-15 19:00:00,Примера,Леганес,Реал Мадрид,1.0,0.0
1,"Сегодня состоится матч по Футболу, в 2 - Дивиз...",1.480,Ф1 (0),1xСтавка,2019-04-20 16:00:00,Сегунда,Альбасете,Лас-Пальмас,0.0,0.0
2,Квалификация Евро. Финляндия против Италии. Ду...,1.720,П2,1xСтавка,2019-09-08 18:45:00,Чемпионат Европы. Квалификация,Финляндия,Италия,0.0,0.0
3,"Арсенал дома, команда сильнее и увереннее",1.444,П1,Winline,2019-09-22 15:30:00,Премьер-лига,Арсенал,Астон Вилла,0.0,0.0
4,барселона проиграла лигу чемпионов она шас буд...,4.100,Ф1 (-2),Леон,2019-05-25 19:00:00,Кубок Испании,Барселона,Валенсия,0.0,0.0


### Команда 1

In [ ]:
for i, text in tqdm(enumerate(subset_russian['result.tsv'].values)):
    if subset_russian['list_direct_match_for_teams1'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([А-Я][а-я]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_ru + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 85:
                    #print(russian['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_russian['list_direct_match_for_teams1'][i] = 1

                    break


In [65]:
success = subset_russian[subset_russian['list_direct_match_for_teams1'] == 1].shape[0]
print(success)
all_vals = subset_russian['list_direct_match_for_teams1'].shape[0]
print(all_vals)
baseline_team1 = success / all_vals
print(baseline_team1)

4413
10000
0.4413


In [151]:
for i, text in tqdm(enumerate(subset_russian['result.tsv'].values)):
    if subset_russian['list_direct_match_for_teams1'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([А-Я][а-я]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_ru + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
#         print(i, good_words)
#         print(subset_russian['team_of_season_1_name_in_russian'][i])
        
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 65 and fuzz.ratio(word, subset_russian['team_of_season_1_name_in_russian'][i]) > 65:
#                     print(subset_russian['team_of_season_1_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_russian['list_direct_match_for_teams1'][i] = 1

                    break












0it [00:00, ?it/s]










6it [00:00,  7.64it/s]










7it [00:02,  1.93it/s]










9it [00:06,  1.06it/s]










11it [00:12,  1.59s/it]










20it [00:15,  1.23s/it]










23it [00:16,  1.05it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy











24it [00:21,  2.02s/it]










25it [00:21,  1.67s/it]










26it [00:24,  1.90s/it]










27it [00:25,  1.60s/it]










28it [00:37,  4.94s/it]










30it [00:39,  3.63s/it]










31it [00:45,  4.36s/it]










32it [00:48,  3.96s/it]










33it [00:56,  5.12s/it]










34it [01:07,  7.15s/it]










37it [01:08,  5.11s/it]










38it [01:09,  3.85s/it]










39it [01:16,  4.82s/it]










40it [01:24,  5.73s

348it [25:43,  5.36s/it]










349it [25:44,  4.08s/it]










350it [25:53,  5.70s/it]










351it [25:55,  4.64s/it]










352it [25:57,  3.73s/it]










354it [26:00,  3.13s/it]










355it [26:11,  5.32s/it]










356it [26:25,  7.90s/it]










359it [26:26,  5.65s/it]










361it [26:35,  5.27s/it]










363it [26:39,  4.32s/it]










365it [27:42, 12.43s/it]










366it [27:45,  9.65s/it]










368it [27:52,  7.77s/it]










369it [27:56,  6.63s/it]










371it [27:57,  4.80s/it]










375it [28:03,  3.82s/it]










377it [28:13,  4.19s/it]










378it [28:19,  4.64s/it]










379it [28:24,  4.80s/it]










381it [28:32,  4.56s/it]










383it [28:40,  4.40s/it]










384it [28:50,  6.10s/it]










385it [28:53,  5.30s/it]










386it [28:56,  4.42s/it]










387it [29:08,  6.77s/it]










389it [29:14,  5.70s/it]










390it [29:17,  4.78s/it]










392it [29:20,  3.80s

708it [53:55,  4.86s/it]










709it [53:59,  4.59s/it]










710it [54:09,  6.15s/it]










711it [54:11,  4.98s/it]










716it [54:16,  3.76s/it]










717it [54:29,  6.79s/it]










718it [54:32,  5.41s/it]










719it [54:39,  6.08s/it]










720it [54:46,  6.29s/it]










725it [54:53,  4.80s/it]










726it [55:01,  5.85s/it]










727it [55:03,  4.73s/it]










731it [55:10,  3.87s/it]










732it [55:15,  4.03s/it]










733it [55:16,  3.33s/it]










734it [55:24,  4.47s/it]










735it [55:26,  3.77s/it]










736it [55:32,  4.48s/it]










738it [55:34,  3.48s/it]










739it [55:40,  4.05s/it]










740it [55:42,  3.49s/it]










741it [55:43,  2.83s/it]










742it [55:48,  3.36s/it]










743it [55:56,  4.88s/it]










744it [55:57,  3.76s/it]










745it [56:01,  3.93s/it]










746it [56:23,  9.35s/it]










747it [56:26,  7.40s/it]










749it [56:29,  5.63s

1024it [1:11:30,  5.61s/it]










1026it [1:11:31,  4.07s/it]










1027it [1:11:40,  5.49s/it]










1028it [1:11:44,  5.19s/it]










1029it [1:11:52,  6.04s/it]










1030it [1:11:58,  6.08s/it]










1032it [1:12:01,  4.70s/it]










1033it [1:12:06,  4.69s/it]










1034it [1:12:07,  3.52s/it]










1036it [1:12:14,  3.56s/it]










1039it [1:12:26,  3.68s/it]










1041it [1:12:29,  3.02s/it]










1043it [1:12:30,  2.31s/it]










1044it [1:12:35,  2.99s/it]










1045it [1:12:42,  4.43s/it]










1046it [1:12:45,  3.84s/it]










1047it [1:12:46,  3.16s/it]










1048it [1:12:51,  3.62s/it]










1050it [1:12:54,  3.01s/it]










1051it [1:13:51, 19.21s/it]










1052it [1:13:52, 13.68s/it]










1053it [1:13:54, 10.08s/it]










1054it [1:14:07, 11.02s/it]










1056it [1:14:09,  7.99s/it]










1057it [1:14:11,  6.26s/it]










1058it [1:14:19,  6.65s/it]










1059it [1:14

1379it [1:27:19,  1.52s/it]










1380it [1:27:23,  2.28s/it]










1381it [1:27:29,  3.27s/it]










1383it [1:27:39,  3.79s/it]










1384it [1:27:41,  3.36s/it]










1385it [1:27:43,  2.81s/it]










1386it [1:27:59,  7.00s/it]










1388it [1:28:14,  7.03s/it]










1389it [1:28:18,  6.34s/it]










1390it [1:28:40, 10.93s/it]










1392it [1:28:41,  7.76s/it]










1394it [1:28:41,  5.55s/it]










1395it [1:28:43,  4.33s/it]










1396it [1:28:44,  3.24s/it]










1397it [1:28:44,  2.47s/it]










1398it [1:28:49,  3.21s/it]










1399it [1:28:54,  3.62s/it]










1401it [1:28:56,  2.90s/it]










1402it [1:28:57,  2.24s/it]










1404it [1:28:58,  1.79s/it]










1405it [1:29:01,  2.01s/it]










1407it [1:29:08,  2.45s/it]










1408it [1:29:16,  4.15s/it]










1411it [1:29:28,  4.12s/it]










1413it [1:29:31,  3.24s/it]










1414it [1:29:32,  2.73s/it]










1416it [1:29

...

8441it [8:20:43,  3.00s/it]










8442it [8:20:47,  3.35s/it]










8443it [8:20:54,  4.33s/it]










8444it [8:20:55,  3.53s/it]










8445it [8:20:58,  3.35s/it]










8446it [8:21:00,  2.94s/it]










8448it [8:21:02,  2.37s/it]










8454it [8:21:13,  2.18s/it]










8455it [8:21:17,  2.81s/it]










8460it [8:21:29,  2.70s/it]










8462it [8:21:50,  4.94s/it]










8463it [8:22:02,  7.10s/it]










8464it [8:22:04,  5.58s/it]










8466it [8:22:10,  4.83s/it]










8467it [8:22:40, 12.45s/it]










8469it [8:22:48,  9.84s/it]










8471it [8:22:50,  7.22s/it]










8473it [8:22:52,  5.37s/it]










8474it [8:23:10,  9.08s/it]










8475it [8:23:22, 10.06s/it]










8476it [8:23:26,  8.19s/it]










8477it [8:23:27,  5.95s/it]










8478it [8:23:28,  4.38s/it]










8480it [8:23:37,  4.55s/it]










8481it [8:23:46,  5.61s/it]










8484it [8:23:49,  4.24s/it]










8485it [8:24

8785it [8:49:17, 17.29s/it]










8787it [8:49:20, 12.60s/it]










8788it [8:49:30, 11.76s/it]










8789it [8:49:32,  8.83s/it]










8790it [8:49:42,  9.07s/it]










8792it [8:49:46,  6.97s/it]










8793it [8:49:49,  5.91s/it]










8795it [8:49:55,  5.06s/it]










8796it [8:50:02,  5.39s/it]










8798it [8:50:04,  4.07s/it]










8799it [8:50:11,  5.04s/it]










8800it [8:50:54, 16.34s/it]










8801it [8:50:54, 11.71s/it]










8802it [8:50:57,  8.83s/it]










8803it [8:50:59,  6.84s/it]










8805it [8:51:00,  4.93s/it]










8808it [8:51:08,  4.26s/it]










8809it [8:51:10,  3.58s/it]










8810it [8:51:45, 12.91s/it]










8811it [8:52:28, 22.06s/it]










8812it [8:52:33, 17.03s/it]










8814it [8:52:35, 12.18s/it]










8815it [8:52:44, 11.24s/it]










8817it [8:52:46,  8.13s/it]










8820it [8:52:48,  5.87s/it]










8822it [8:52:58,  5.74s/it]










8823it [8:53

9110it [9:15:39,  1.73s/it]










9111it [9:15:42,  2.02s/it]










9112it [9:15:46,  2.50s/it]










9114it [9:15:47,  2.04s/it]










9115it [9:15:52,  2.70s/it]










9116it [9:15:55,  2.79s/it]










9117it [9:15:57,  2.71s/it]










9118it [9:15:59,  2.56s/it]










9120it [9:16:00,  1.94s/it]










9121it [9:16:05,  2.57s/it]










9124it [9:16:09,  2.22s/it]










9125it [9:16:11,  2.23s/it]










9130it [9:16:13,  1.67s/it]










9131it [9:16:17,  2.34s/it]










9132it [9:16:18,  1.92s/it]










9134it [9:16:26,  2.66s/it]










9135it [9:16:34,  4.02s/it]










9140it [9:16:37,  3.00s/it]










9143it [9:16:41,  2.50s/it]










9144it [9:16:42,  2.07s/it]










9146it [9:16:48,  2.37s/it]










9149it [9:16:53,  2.14s/it]










9150it [9:16:58,  3.02s/it]










9151it [9:16:59,  2.62s/it]










9152it [9:17:02,  2.61s/it]










9153it [9:17:04,  2.51s/it]










9155it [9:17

9437it [9:34:31,  2.84s/it]










9439it [9:34:38,  3.00s/it]










9440it [9:34:44,  3.80s/it]










9441it [9:34:48,  3.80s/it]










9442it [9:34:54,  4.58s/it]










9443it [9:34:56,  3.83s/it]










9444it [9:35:04,  5.10s/it]










9445it [9:35:05,  3.85s/it]










9446it [9:35:10,  4.07s/it]










9447it [9:35:14,  4.12s/it]










9448it [9:35:17,  3.79s/it]










9449it [9:35:20,  3.56s/it]










9450it [9:35:21,  2.97s/it]










9451it [9:35:28,  4.18s/it]










9452it [9:35:36,  5.32s/it]










9453it [9:35:41,  5.03s/it]










9454it [9:35:46,  5.17s/it]










9456it [9:35:49,  4.00s/it]










9457it [9:35:50,  3.28s/it]










9458it [9:35:56,  4.03s/it]










9460it [9:36:16,  5.79s/it]










9463it [9:36:23,  4.71s/it]










9467it [9:36:30,  3.88s/it]










9468it [9:36:33,  3.58s/it]










9469it [9:37:28, 18.91s/it]










9470it [9:37:32, 14.33s/it]










9471it [9:37

9754it [9:55:11,  5.72s/it]










9755it [9:56:12, 22.26s/it]










9756it [9:56:44, 25.14s/it]










9757it [9:56:49, 19.06s/it]










9758it [9:57:00, 16.78s/it]










9761it [9:57:03, 12.01s/it]










9762it [9:57:16, 12.41s/it]










9763it [9:57:20,  9.74s/it]










9764it [9:57:27,  9.14s/it]










9767it [9:57:33,  7.00s/it]










9768it [9:57:41,  7.22s/it]










9769it [9:57:42,  5.37s/it]










9770it [9:57:43,  4.18s/it]










9771it [9:57:44,  3.14s/it]










9773it [9:57:48,  2.72s/it]










9774it [9:57:49,  2.17s/it]










9775it [9:57:57,  4.10s/it]










9776it [9:57:59,  3.48s/it]










9778it [9:58:05,  3.23s/it]










9779it [9:58:05,  2.51s/it]










9780it [9:58:09,  2.89s/it]










9782it [9:58:13,  2.53s/it]










9783it [9:58:19,  3.66s/it]










9784it [9:58:22,  3.58s/it]










9785it [9:58:23,  2.80s/it]










9786it [9:58:25,  2.45s/it]










9787it [9:58

In [152]:
success = subset_russian[subset_russian['list_direct_match_for_teams1'] == 1].shape[0]
print(success)
all_vals = subset_russian['list_direct_match_for_teams1'].shape[0]
print(all_vals)
baseline_team1 = success / all_vals
print(baseline_team1)

6457
10000
0.6457


### Команда 2 

In [ ]:
for i, text in tqdm(enumerate(subset_russian['result.tsv'].values)):
    if subset_russian['list_direct_match_for_teams2'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([А-Я][а-я]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_ru + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 85 and subset_russian['team_of_season_2_name_in_russian'][i] == word:
                    #print(russian['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_russian['list_direct_match_for_teams2'][i] = 1

                    break


In [66]:
success = subset_russian[subset_russian['list_direct_match_for_teams2'] == 1].shape[0]
print(success)
all_vals = subset_russian['list_direct_match_for_teams2'].shape[0]
print(all_vals)
baseline_team2 = success / all_vals
print(baseline_team2)

4144
10000
0.4144


In [153]:
for i, text in tqdm(enumerate(subset_russian['result.tsv'].values)):
    if subset_russian['list_direct_match_for_teams2'][i] == 1:
        continue
    else:
        #print(i)
        capital_words_ru = re.findall(r'([А-Я][а-я]+)', text)
        quotes = re.findall(r'\"(.+?)\"', text)
        quotes2 = re.findall(r'«(.+?)»', text)
        quotes3 = re.findall(r'\'(.+?)\'', text)
        search_words = capital_words_ru + quotes + quotes2 + quotes3
        search_words = set(search_words)
        good_words = [word for word in search_words if word.lower() not in newlist_set] 
        #print(good_words)
    
        for word in good_words:
            for team in unique_teams:
                if fuzz.ratio(word, team) > 65 and fuzz.ratio(word, subset_russian['team_of_season_2_name_in_russian'][i]) > 65:                    #print(russian['team_of_season_2_name_in_russian'][i], word, team, fuzz.ratio(word, team))
                    subset_russian['list_direct_match_for_teams2'][i] = 1

                    break












0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy











1it [00:06,  6.27s/it]










2it [00:21,  8.99s/it]










3it [00:28,  8.43s/it]










4it [00:29,  6.16s/it]










6it [00:30,  4.42s/it]










7it [00:31,  3.49s/it]










9it [00:34,  2.91s/it]










11it [00:38,  2.63s/it]










12it [00:42,  3.07s/it]










17it [00:54,  2.85s/it]










20it [00:56,  2.22s/it]










21it [01:00,  2.58s/it]










22it [01:04,  3.13s/it]










23it [01:05,  2.42s/it]










24it [01:10,  3.39s/it]










25it [01:12,  2.79s/it]










26it [01:15,  2.79s/it]










27it [01:16,  2.26s/it]










28it [01:29,  5.61s/it]










30it [01:30,  4.10s/it]

332it [21:00,  1.80s/it]










333it [21:01,  1.56s/it]










334it [21:04,  1.82s/it]










337it [21:14,  2.32s/it]










339it [21:31,  4.15s/it]










340it [21:37,  4.58s/it]










342it [21:41,  3.83s/it]










343it [21:48,  4.99s/it]










344it [21:53,  4.84s/it]










346it [21:55,  3.64s/it]










347it [21:56,  2.84s/it]










348it [22:04,  4.44s/it]










349it [22:05,  3.34s/it]










350it [22:12,  4.69s/it]










351it [22:15,  4.03s/it]










352it [22:17,  3.38s/it]










354it [22:20,  2.77s/it]










355it [22:29,  4.90s/it]










359it [22:30,  3.49s/it]










361it [22:41,  4.05s/it]










363it [22:45,  3.45s/it]










365it [23:41, 10.86s/it]










366it [23:45,  8.71s/it]










367it [23:54,  8.74s/it]










368it [24:01,  8.33s/it]










369it [24:05,  6.88s/it]










370it [24:14,  7.70s/it]










373it [24:19,  5.87s/it]










377it [24:29,  4.87s

675it [44:45, 14.45s/it]










677it [44:50, 10.92s/it]










681it [44:56,  8.11s/it]










682it [44:57,  5.93s/it]










683it [45:01,  5.34s/it]










684it [45:04,  4.73s/it]










685it [45:07,  4.09s/it]










686it [45:09,  3.58s/it]










688it [45:15,  3.36s/it]










693it [45:18,  2.54s/it]










694it [45:34,  6.35s/it]










695it [45:38,  5.89s/it]










697it [45:39,  4.26s/it]










699it [45:40,  3.08s/it]










700it [45:42,  2.74s/it]










701it [45:50,  4.28s/it]










702it [45:50,  3.21s/it]










703it [45:56,  3.94s/it]










704it [46:03,  4.75s/it]










705it [46:08,  4.91s/it]










706it [46:15,  5.68s/it]










707it [46:17,  4.55s/it]










708it [46:21,  4.38s/it]










709it [46:25,  4.10s/it]










711it [46:27,  3.25s/it]










713it [46:35,  3.46s/it]










716it [46:38,  2.70s/it]










717it [46:49,  5.13s/it]










720it [46:55,  4.22s

1012it [1:00:52,  1.49s/it]










1013it [1:00:54,  1.68s/it]










1015it [1:01:02,  2.31s/it]










1017it [1:01:07,  2.37s/it]










1018it [1:01:13,  3.56s/it]










1019it [1:01:19,  4.20s/it]










1020it [1:01:20,  3.11s/it]










1021it [1:01:25,  3.80s/it]










1022it [1:01:26,  2.86s/it]










1023it [1:01:42,  6.79s/it]










1024it [1:01:43,  5.29s/it]










1025it [1:01:50,  5.52s/it]










1026it [1:01:51,  4.39s/it]










1027it [1:02:02,  6.24s/it]










1028it [1:02:07,  5.80s/it]










1029it [1:02:15,  6.53s/it]










1030it [1:02:21,  6.55s/it]










1031it [1:02:38,  9.51s/it]










1032it [1:02:41,  7.59s/it]










1036it [1:02:50,  5.99s/it]










1039it [1:03:03,  5.48s/it]










1041it [1:03:07,  4.39s/it]










1043it [1:03:08,  3.27s/it]










1045it [1:03:15,  3.33s/it]










1046it [1:03:16,  2.79s/it]










1047it [1:03:18,  2.46s/it]










1048it [1:03

1336it [1:19:04,  4.41s/it]










1338it [1:19:09,  3.83s/it]










1339it [1:19:13,  3.67s/it]










1341it [1:19:18,  3.40s/it]










1342it [1:19:19,  2.62s/it]










1343it [1:19:19,  2.01s/it]










1346it [1:19:23,  1.74s/it]










1347it [1:19:25,  1.75s/it]










1349it [1:19:25,  1.33s/it]










1353it [1:19:30,  1.26s/it]










1354it [1:19:32,  1.54s/it]










1355it [1:19:36,  2.30s/it]










1361it [1:19:37,  1.66s/it]










1362it [1:19:39,  1.88s/it]










1365it [1:19:47,  2.13s/it]










1366it [1:19:54,  3.53s/it]










1367it [1:19:58,  3.74s/it]










1370it [1:20:13,  4.11s/it]










1371it [1:20:19,  4.65s/it]










1373it [1:20:20,  3.39s/it]










1374it [1:20:21,  2.59s/it]










1375it [1:20:28,  3.85s/it]










1376it [1:20:29,  2.95s/it]










1378it [1:20:33,  2.67s/it]










1380it [1:20:35,  2.21s/it]










1381it [1:20:39,  2.77s/it]










1383it [1:20

...

8473it [7:47:44,  4.50s/it]










8477it [7:47:45,  3.26s/it]










8478it [7:47:46,  2.52s/it]










8480it [7:47:53,  2.78s/it]










8481it [7:47:57,  3.17s/it]










8482it [7:48:06,  4.95s/it]










8484it [7:48:09,  3.86s/it]










8487it [7:48:09,  2.78s/it]










8488it [7:48:13,  2.96s/it]










8489it [7:48:25,  5.88s/it]










8490it [7:48:29,  5.10s/it]










8492it [7:48:40,  5.21s/it]










8493it [7:48:48,  6.11s/it]










8494it [7:48:50,  4.99s/it]










8495it [7:48:52,  4.09s/it]










8496it [7:49:01,  5.44s/it]










8498it [7:49:04,  4.28s/it]










8499it [7:49:14,  5.96s/it]










8500it [7:49:15,  4.45s/it]










8502it [7:49:17,  3.49s/it]










8506it [7:49:20,  2.65s/it]










8507it [7:49:24,  3.05s/it]










8508it [7:49:29,  3.48s/it]










8509it [7:49:32,  3.48s/it]










8511it [7:49:48,  4.87s/it]










8512it [7:49:49,  3.66s/it]










8513it [7:49

8801it [8:06:56,  9.02s/it]










8802it [8:06:58,  6.80s/it]










8803it [8:06:58,  5.04s/it]










8804it [8:07:03,  5.02s/it]










8805it [8:07:04,  3.74s/it]










8807it [8:07:05,  2.76s/it]










8808it [8:07:12,  4.06s/it]










8809it [8:07:13,  3.08s/it]










8811it [8:07:38,  5.93s/it]










8812it [8:07:42,  5.17s/it]










8814it [8:07:42,  3.75s/it]










8815it [8:07:47,  3.98s/it]










8817it [8:07:49,  3.02s/it]










8818it [8:07:54,  3.78s/it]










8819it [8:07:55,  2.95s/it]










8821it [8:07:58,  2.48s/it]










8822it [8:08:03,  3.38s/it]










8823it [8:08:10,  4.47s/it]










8825it [8:08:15,  3.82s/it]










8826it [8:08:16,  2.92s/it]










8828it [8:08:23,  3.05s/it]










8830it [8:08:25,  2.56s/it]










8831it [8:08:29,  3.03s/it]










8832it [8:08:35,  3.69s/it]










8833it [8:08:36,  3.03s/it]










8835it [8:08:38,  2.41s/it]










8837it [8:08

9130it [8:26:19,  2.87s/it]










9131it [8:26:25,  3.57s/it]










9132it [8:26:26,  2.77s/it]










9134it [8:26:34,  3.22s/it]










9135it [8:26:42,  4.63s/it]










9140it [8:26:44,  3.39s/it]










9143it [8:26:50,  2.89s/it]










9144it [8:26:51,  2.28s/it]










9147it [8:26:51,  1.69s/it]










9149it [8:26:56,  1.79s/it]










9151it [8:26:57,  1.50s/it]










9152it [8:27:01,  2.12s/it]










9153it [8:27:03,  2.25s/it]










9155it [8:27:05,  1.82s/it]










9156it [8:27:10,  2.67s/it]










9158it [8:27:12,  2.25s/it]










9159it [8:27:19,  3.63s/it]










9162it [8:27:20,  2.67s/it]










9163it [8:27:50, 10.81s/it]










9164it [8:27:55,  9.11s/it]










9165it [8:27:58,  7.10s/it]










9166it [8:27:59,  5.24s/it]










9167it [8:27:59,  3.91s/it]










9168it [8:28:04,  4.17s/it]










9171it [8:28:08,  3.27s/it]










9172it [8:28:15,  4.41s/it]










9173it [8:28

9457it [8:44:09,  2.78s/it]










9458it [8:44:13,  3.14s/it]










9460it [8:44:28,  4.42s/it]










9463it [8:44:36,  3.91s/it]










9465it [8:44:44,  4.04s/it]










9467it [8:44:52,  3.90s/it]










9468it [8:44:53,  3.25s/it]










9469it [8:45:40, 16.41s/it]










9470it [8:45:43, 12.23s/it]










9472it [8:45:47,  9.22s/it]










9473it [8:45:52,  7.77s/it]










9474it [8:46:01,  8.22s/it]










9475it [8:46:02,  6.20s/it]










9476it [8:46:08,  5.87s/it]










9478it [8:46:13,  4.94s/it]










9479it [8:46:21,  5.74s/it]










9480it [8:46:24,  4.97s/it]










9481it [8:46:29,  5.09s/it]










9482it [8:46:31,  4.20s/it]










9484it [8:46:34,  3.29s/it]










9485it [8:46:39,  3.78s/it]










9487it [8:46:43,  3.34s/it]










9489it [8:46:49,  3.16s/it]










9490it [8:46:54,  3.74s/it]










9491it [8:46:55,  2.89s/it]










9492it [8:46:56,  2.48s/it]










9493it [8:47

9767it [9:04:16,  4.26s/it]










9770it [9:04:17,  3.08s/it]










9773it [9:04:21,  2.56s/it]










9774it [9:04:21,  1.85s/it]










9775it [9:04:27,  3.15s/it]










9776it [9:04:29,  2.65s/it]










9778it [9:04:33,  2.48s/it]










9779it [9:04:33,  1.96s/it]










9780it [9:04:37,  2.42s/it]










9782it [9:04:39,  2.04s/it]










9783it [9:04:45,  3.14s/it]










9784it [9:04:47,  2.95s/it]










9785it [9:04:48,  2.34s/it]










9786it [9:04:49,  1.91s/it]










9787it [9:04:52,  2.30s/it]










9788it [9:04:58,  3.25s/it]










9789it [9:04:59,  2.47s/it]










9791it [9:05:05,  2.70s/it]










9793it [9:05:10,  2.57s/it]










9794it [9:05:18,  4.17s/it]










9796it [9:05:20,  3.34s/it]










9797it [9:05:22,  2.77s/it]










9799it [9:05:46,  5.51s/it]










9801it [9:05:53,  4.91s/it]










9802it [9:05:55,  4.22s/it]










9804it [9:05:59,  3.58s/it]










9805it [9:06

In [154]:
success = subset_russian[subset_russian['list_direct_match_for_teams2'] == 1].shape[0]
print(success)
all_vals = subset_russian['list_direct_match_for_teams2'].shape[0]
print(all_vals)
baseline_team2 = success / all_vals
print(baseline_team2)

6209
10000
0.6209


In [155]:
subset_russian.to_csv(r'C:\Users\User\Desktop\project\fuzzylistsnew20106.csv', index = False)

### Лига

In [83]:
leagues_match = np.zeros(subset_russian.shape[0])

In [84]:
for i, text in tqdm(enumerate(subset_russian['result.tsv'].values)):
    #print(i)
    capital_words_ru = re.findall(r'([А-Я][а-я]+)', text)
    quotes = re.findall(r'\"(.+?)\"', text)
    quotes2 = re.findall(r'«(.+?)»', text)
    quotes3 = re.findall(r'\'(.+?)\'', text)
    search_words = capital_words_ru + quotes + quotes2 + quotes3
    search_words = set(search_words)
    good_words = [word for word in search_words if word.lower() not in newlist_set] 
    #print(good_words)
    
    for word in good_words:
        for league in unique_leagues:
            if fuzz.ratio(word, league) > 50 and fuzz.ratio(word, subset_russian['league_name_in_russian'][i]) > 50:
                print(subset_russian['league_name_in_russian'][i], word, league, fuzz.ratio(word, league))
                #print('yes')
                leagues_match[i] = 1

                break










0it [00:00, ?it/s]








1it [00:00,  3.42it/s]








2it [00:00,  3.84it/s]








3it [00:00,  4.61it/s]








5it [00:01,  4.47it/s]








7it [00:01,  5.60it/s]

Лига чемпионов УЕФА Чемпионов Чемпионшип 74











9it [00:01,  4.68it/s]








10it [00:02,  2.80it/s]








11it [00:02,  3.05it/s]








13it [00:03,  3.41it/s]








14it [00:03,  2.88it/s]








15it [00:03,  3.34it/s]








17it [00:03,  4.38it/s]








18it [00:04,  4.29it/s]








19it [00:04,  5.14it/s]

Лига чемпионов УЕФА Чемпионов Чемпионшип 74











20it [00:04,  5.30it/s]








21it [00:04,  4.54it/s]








22it [00:05,  4.45it/s]








23it [00:05,  4.87it/s]








24it [00:06,  2.22it/s]








25it [00:06,  2.62it/s]

...










9981it [46:37,  5.32it/s]








9982it [46:37,  5.56it/s]








9984it [46:37,  5.83it/s]








9985it [46:38,  5.35it/s]








9988it [46:38,  6.67it/s]








9990it [46:39,  4.65it/s]








9991it [46:40,  1.96it/s]








9992it [46:40,  2.12it/s]








9993it [46:41,  2.34it/s]








9994it [46:41,  2.75it/s]

Кубок Америки Америки Кубок Америки 70
Кубок Америки Кубке Кубок Америки 56











9995it [46:41,  2.69it/s]








9996it [46:41,  2.68it/s]








9997it [46:42,  2.34it/s]

J-Лига Лиги Лига 1 60











10000it [46:43,  3.57it/s]


In [85]:
subset_russian['list_fuzzy_match_for_league'] = leagues_match

In [86]:
success = subset_russian[subset_russian['list_fuzzy_match_for_league'] == 1].shape[0]
print(success)
all_vals = subset_russian['list_fuzzy_match_for_league'].shape[0]
print(all_vals)
baseline_league = success / all_vals
print(baseline_league)

1642
10000
0.1642


In [87]:
subset_russian.to_csv(r'C:\Users\User\Desktop\project\fuzzylists0306_withleagues.csv', index = False)